## Imports

In [1]:
# Import the usual modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Import Olist data
from olist.data import Olist
from olist.order import Order

# Olist's Net Promoter Score (NPS) 🔥

The **Net Promoter Score (NPS)** of a service answers the following question:

> How likely is it that you would recommend our company/product/service to a friend or colleague?

For a service rated between 1 and 5 stars, like Olist, we can **classify customers into three categories** based on their answers:
- ✅ **Promoters**: customers who answered  with a score of 5
- 😴 **Passive**: customers who answered with a score of 4 
- 😡 **Detractors**: customers who answered with a score between 1 and 3 (inclusive)

<br>

👉 NPS is computed by subtracting the percentage of customers who are **detractors** from the percentage of customers who are **promoters**.

> NPS  
= % Promoters - % Detractors   
= (# Promoter - # Detractors) / # Reviews  
= (# 5 stars - # <4 stars) / # Reviews

## Computing the Overall NPS Score of Olist

In [2]:
data = Olist().get_data()
orders = Order().get_training_data()

❓Create a function that converts `review_score` into `nps_class`. `nps_class` should be a **classification** depending on the `review_score`, so there are 3 possibilities:

- `review_score` is **5** 👉 `nps_class` is **1** (promoter)
- `review_score` is **4** 👉 `nps_class` is **0** (passive)
- `review_score` is **3** or less 👉 `nps_class` is **-1** (detractor)

In [5]:
def promoter_score(review_score):
 
    score = 0

    if review_score == 5:
        score = 1
    if review_score < 4:
        score = -1

    return score

orders.review_score.map(promoter_score)


0        0
1        0
2        1
3        1
4        1
        ..
96356    1
96357    0
96358    1
96359   -1
96360    1
Name: review_score, Length: 96353, dtype: int64

💡Let's try to rewrite this function into a single line of code that achieves the same result 😏

There are **several** ways to do it! Let's look at some of them, then we can compare their execution times to that of our function to see which one is more efficient ⏱️

Two general principles when it comes to programming/coding are:
- `KISS`: **K**eep **I**t **S**imple and **S**mart
- `DRY`: **D**on't **R**epeat **Y**ourself 😉

<details>
    <summary>💡Hint</summary>

Use the following methods and use `%time` to compare their execution times:
- `.apply()` with the function you wrote above
- `.map()` or `.apply()` with a `lambda` function
- `.loc[]` with boolean indexing
- `np.select()` with matching conditions

</details>    

In [6]:
# YOUR CODE HERE
%time
orders['promoter_class'] = orders.review_score.apply(promoter_score)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


In [7]:
%time
orders['promoter_class'] = orders.review_score.map(promoter_score)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.39 µs


In [8]:
%time
orders['promoter_class'] = orders.review_score.map({5: 1, 4: 0, 3: -1, 2: -1, 1: -1})

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs


**A Note About `.apply()`**

Consider the following examples:

```python
df.apply(lambda col: col.max(), axis = 0)
df.apply(lambda row: row['A'] + row['B'], axis = 1)
```

These operations look similar because they both use `.apply()`, but one is much slower than the other. The data layout for Pandas DataFrames is **column-major** (read more [here](https://en.wikipedia.org/wiki/Row-_and_column-major_order)), which means that column-wise operations are always going to be faster than row-wise operations. The second example above uses `axis=1`, making it a row-wise operation, which would be more appropriate for **row-major** data layouts such as NumPy arrays.

For small amounts of data, this difference is irrelevant, but when you start working with huge datasets this will probably make a big difference. For big datasets, you're likely to notice that using `.loc[]`, `np.select()` or `np.apply_along_axis()` will run faster on Pandas DataFrames when applying a function on every row.

It's always good to understand how your data is stored before you access it!

👇 Now that you have the different promoter scores, you can compute `Olist's NPS`.

In [9]:
# YOUR CODE HERE
nps = orders.review_score.map({5: 1, 4: 0, 3: -1, 2: -1, 1: -1}).sum() / orders.review_score.count()

In [10]:
f'NPS Score: {nps * 100:.1f}%'

'NPS Score: 38.1%'

## NPS per Customer State

👇 Here is the part of Olist's DB schema that is relevant for this section, to help you have an overview of things.

<img src="https://wagon-public-datasets.s3-eu-west-1.amazonaws.com/04-Decision-Science/02-Statistical-Inference/olist_schema.png" width=750>

### What is the average review score per state?

❓First, create the dataset required for computation

In [11]:
# YOUR CODE HERE
merge = data['orders']\
.merge(data['order_reviews'], on='order_id')\
.merge(data['customers'], on='customer_id')

merge

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,8d5266042046a06655c8db133d120ba5,4,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,af07308b275d755c9edb36a90c618231,47813,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,e73b67b67587f7644d5bd1a52deb1b01,5,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,359d03e676b3c069f62cadba8dd3f6e8,5,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00,2017-12-05 19:21:58,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,e50934924e227544ba8246aeb3770dd4,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 13:02:51,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99219,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,e262b3f92d1ce917aa412a9406cf61a6,5,NaN,NaN,2017-03-22 00:00:00,2017-03-23 11:02:08,6359f309b166b0196dbf7ad2ac62bb5a,12209,sao jose dos campos,SP
99220,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,29bb71b2760d0f876dfa178a76bc4734,4,NaN,So uma peça que veio rachado mas tudo bem rs,2018-03-01 00:00:00,2018-03-02 17:50:01,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP
99221,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,371579771219f6db2d830d50805977bb,5,NaN,Foi entregue antes do prazo.,2017-09-22 00:00:00,2017-09-22 23:10:57,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA
99222,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,8ab6855b9fe9b812cd03a480a25058a1,2,NaN,Foi entregue somente 1. Quero saber do outro p...,2018-01-26 00:00:00,2018-01-27 09:16:56,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ


👉 Now, we can aggregate this dataset per  `customer_state` using any aggregation method of our choice :)

❓ Let's start with the average review score: compute the average `review_score` per `customer_state`.

*Hints:* try to tackle this question using three different methods:
- with `.mean()`
- then with `.apply()`
- and eventually the `.agg()`

In [12]:
# YOUR CODE HERE
merge.groupby(['customer_state'])['review_score'].mean().head()

customer_state
AC    4.049383
AL    3.751208
AM    4.183673
AP    4.194030
BA    3.860888
Name: review_score, dtype: float64

In [16]:
# YOUR CODE HERE
merge.groupby(['customer_state'])['review_score'].apply(np.mean).head()

customer_state
AC    4.049383
AL    3.751208
AM    4.183673
AP    4.194030
BA    3.860888
Name: review_score, dtype: float64

In [15]:
# YOUR CODE HERE
merge.groupby(['customer_state']).agg({'review_score': np.mean})\
.rename(columns={'review_score': 'mean_review_score'})\
.head()

,mean_review_score
customer_state,
AC,4.049383
AL,3.751208
AM,4.183673
AP,4.194030
BA,3.860888


🤩 `.agg()` is much more flexible than the other methods, push it further!

In [14]:
# YOUR CODE HERE

### NPS per State

❓Now, it is time to create a 🔥 **custom aggregation function** to compute the `NPS per customer_state` directly.

1️⃣ Create your `nps` function

2️⃣ Try to debug it using the `breakpoint()` debugger within your function to understand clearly what objects you are manipulating

<br>

💡 *PS.:* always **cleanly** exit your debugger by typing `exit` when inside the debugging session, otherwise you will have to restart your Notebook!

In [18]:
# YOUR CODE HERE
def nps(series):
    return series.map(promoter_score).sum() / series.count()

👉 Now, use your `nps` function to compute the `NPS per customer_state`.

In [19]:
# YOUR CODE HERE
merge.groupby(['customer_state']).agg({
    'review_score': nps,
}).head()

,review_score
customer_state,
AC,0.296296
AL,0.166667
AM,0.414966
AP,0.313433
BA,0.199881


Again, instead of using this function, try to do the same task in one line of code, remember the `KISS` principle? 😉

In [20]:
# YOUR CODE HERE
merge.groupby(['customer_state'])['review_score'].apply(nps)

customer_state
AC    0.296296
AL    0.166667
AM    0.414966
AP    0.313433
BA    0.199881
CE    0.207675
DF    0.333799
ES    0.311012
GO    0.305830
MA    0.156836
MG    0.372989
MS    0.374309
MT    0.343300
PA    0.205579
PB    0.306968
PE    0.312272
PI    0.250509
PR    0.398769
RJ    0.253349
RN    0.358921
RO    0.301587
RR    0.043478
RS    0.368959
SC    0.332597
SE    0.206304
SP    0.397481
TO    0.326165
Name: review_score, dtype: float64

# Cheat Sheet for `map`, `apply`, `applymap` and `groupby`

```python
# MAP (for Series)
series.map(function) 
series.map({mapping dict})

# APPLY (for DataFrame)
df.apply(lambda col: col.max(), axis = 0)     # default axis
df.apply(lambda row: row[‘A’] + row[‘B’], axis = 1)

df.applymap(my_funct_for_indiv_elements)
df.applymap(lambda x: '%.2f' % x)
```

```python
## GROUPBY
group = df.groupby('col_A')

group.mean()
group.apply(np.mean)
group.agg({
    col_A: ['mean', np.sum],
    col_B: my_custom_sum,
    col_B: lambda s: my_custom_sum(s)
})

group.apply(custom_mean_function)
```

[Introduction to Pandas' `apply`, `applymap` and `map` - Towards Data Science](https://towardsdatascience.com/introduction-to-pandas-apply-applymap-and-map-5d3e044e93ff)